In [1]:
from sklearn.svm import SVC, LinearSVC

In [ ]:
class SVMClassifier():

    def __init__(self, C=1.0, kernel='rbf', probability=False, class_weight=None, verbose=True, max_iter=-1):
        
        if kernel is not None:
            print('Kernel : ', kernel)
            self.model = SVC(C=C, kernel=kernel, probability=probability,
                         class_weight=class_weight, verbose=verbose, max_iter=max_iter)
        else:
            print(' Linear Kernel with l2 peanlization ')
            self.model = LinearSVC(C=C,verbose=verbose)
            
    def fit(self, X, Y, max_itr = 10, W=None):
        kf = KFold(n_splits=max_itr)
        X, Y = shuffle(X, Y)
        
        training_scores = []
        validation_scores = []
        #for itr in range(0, max_itr):
        for train_index, test_index in kf.split(X):
            
            X_train, X_valid = X[train_index], X[test_index]
            Y_train, Y_valid = Y[train_index], Y[test_index]
            
            self.model.fit(X_train, Y_train, sample_weight=W)
            
            training_scores.append(self.score(X_train, Y_train))
            validation_scores.append(self.score(X_valid, Y_valid))
            
        print('SVM training done-----')
        return training_scores, validation_scores

    def predict(self, x):
        return self.model.predict(x)

    def score(self, x, y, w=None):
        score = self.model.score(x, y, sample_weight=w)
        
        return score
    
    def get_params(self, deep=True):
        return self.model.get_params(deep)


In [ ]:
def SVM_model(X_matrix, Y, x_matrix, y, C=1, kernel = 'rbf', max_itr = 10, plot=False, words=False, filename = None):

    svm = SVMClassifier(C = C, kernel = kernel)
    training_scores, validation_scores = svm.fit(X_matrix, Y, max_itr = max_itr)

    # Plot scores
    plot_scores(max_itr, training_scores, validation_scores)
    
    if plot:
        file = '{}/{}-boundary.png'.format(result_folder, filename)
        plot_boundary(X_matrix, Y, svm, filename = file, h=1)
    
    print('\n----------------------------Parameters---------------------------------------------\n')
#     training_score = svm.score(X_matrix, Y)
#     print('Trainign Accuracy score : ', training_score)
#     print('\n------------------------------------------------------------------------------------\n')
          
    y_trainig_hat = svm.predict(X_matrix)
#     print('precision score ', precision_score(Y, y_trainig_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('f1 score ', f1_score(Y, y_trainig_hat))
    
    train_metric = metric_score(Y, y_trainig_hat, type='Training')

#     test_score = svm.score(x_matrix, y)
#     print('Test Accuracy score : ', test_score)
#     print('\n------------------------------------------------------------------------------------\n')
          
    y_test_hat = svm.predict(x_matrix)
#     print('precision score ', precision_score(y, y_test_hat))
#     print('\n------------------------------------------------------------------------------------\n')
#     print('f1 score ', f1_score(y, y_test_hat))
    
    test_metric = metric_score(y, y_test_hat, type='Test')

    if filename is not None:
        pd_best = pd.DataFrame({"best_params" : {'model': svm.model, 'model__alpha': C}})
        save_results_html(filename, pd.concat([pd.concat([train_metric, test_metric], axis=1, sort=False), pd_best], keys=['Score', 'Model'], axis=1, sort=False))
    
    if words:
        print('------- Top words ---------')
        print_words(svm.model.dual_coef_.T)

In [ ]:
def SVM_model_selection(X_matrix, Y, x_matrix, y, params, file = None, threads=1):
    
    GridSearchModel(X_matrix, Y, x_matrix, y, params, filename=file, threads=threads)